# Chap03 - 딕셔너리와 집합

> 프로그램 코드 안에서 명시적으로 딕셔너리 사용하고 있지 않더라도, 모든 파이썬 프로그램에서는 여러 딕셔너리가 동시에 활동하고 있다.
-A.M 커츨링-

파이썬의 딕셔너리(dictionary)와 집합(set)은 [**해시 테이블**](https://ko.wikipedia.org/wiki/%ED%95%B4%EC%8B%9C_%ED%85%8C%EC%9D%B4%EB%B8%94)을 이용해 구현되었다. 

이번 Chap03 에서 다룰 내용은 다음과 같다.

 - Common dictionary methods
 - Special handling for missing keys
 - Variations of dict in the standard library
 - The set and frozenset types
 - How hash tables work
 - Implications of hash tables: key type limitations, unpredictable ordering etc.

## 3.1 Generic Mapping Types

`collections.abc` 모듈은 `dict`(딕셔너리) 및 유사한 자료형의 인터페이스를 정의하는 `Mapping` 및 `MutalbelMapping` 추상 클래스(ABC)를 제공한다.

![](./images/3-1.PNG)

아래의 예제에서도 확인할 수 있듯이, `isinstance()` 함수를 이용해 딕셔너리(`dict`)는 `collections.abc.Mapping`의 자료형임을 알 수 있다.

In [1]:
import collections

my_dict = {}
isinstance(my_dict, collections.abc.Mapping)

True

파이썬에서 제공하는 매핑형(mapping type)은 모두 `dict`를 이용해 구현되기 때문에, 키(key)가 **해시 가능([hashable](https://docs.python.org/3/glossary.html#term-hashable))**해야 한다.

> hashable이란 변하지 않는 해시값을 가지고 있고 다른 객체와 비교할 수 있으면, 이 객체를 hashable이라고 한다.

위의 설명에서도 유추할 수 있듯이 불변형인 `str, byte, tuple, ...`은 hashable 하다. 

아래의 예제코드에서 `hash()`함수는 해당 객체의 해시값을 반환해준다.

In [2]:
tt = (1, 2, (30, 40))
hash(tt)

8027212646858338501

아래의 예제코드에서 에러가 나는 이유는 튜플(tuple)안에 가변 시퀀스인 리스트(`list`)가 들어있기 때문이다.

In [3]:
t1 = (1, 2, [30, 40])
hash(t1)

TypeError: unhashable type: 'list'

마찬가지로 `set`(집합) 또한 가변 시퀀스이기 때문에, `hash()`함수를 적용하면 에러가 난다.

In [6]:
tf = (1, 2, set([30, 40]))
hash(tf)

TypeError: unhashable type: 'set'

`set`과 같은 기능을 하는 [`frozenset`](https://docs.python.org/ko/3.6/library/stdtypes.html?highlight=frozenset#frozenset)은 hashable 하다.

In [7]:
tf2 = (1, 2, frozenset([30, 40]))
hash(tf2)

985328935373711578

파이썬 [공식문서](https://docs.python.org/ko/3/library/stdtypes.html#mapping-types-dict)에서도 확인할 수 있듯이, 아래와 같이 다양한 방식으로 딕셔너리를 구현할 수 있다.

In [14]:
a = dict(one=1, two=2, three=3)
b = {'one': 1, 'two': 2, 'three': 3}
c = dict(zip(['one', 'two', 'three'], [1, 2, 3]))
d = dict([('two', 2), ('one', 1), ('three', 3)])
e = dict({'three': 3, 'one': 1, 'two': 2})

print(a == b == c == d == e)
print(a)

True
{'one': 1, 'two': 2, 'three': 3}


## 3.2 딕셔너리 컴프리헨션

[2장-시퀀스](http://excelsior-cjh.tistory.com/164)에서 살펴본 리스트 컴프리헨션과 동일하게 딕셔너리 또한 **딕셔너리 컴프리헨션(dict comprehension)**을 이용해 딕셔너리 객체를 만들 수 있다.

In [17]:
DIAL_CODES = [(82, 'Korea'),
              (86, 'China'),
              (91, 'India'),
              (1, 'United States'),
              (62, 'Indonesia'),
              (55, 'Brazil'),
              (92, 'Pakistan'),
              (880, 'Bangladesh'),
              (234, 'Nigeria'),
              (7, 'Russia')]

country_code = {country.upper(): code for code, country in DIAL_CODES}
print(country_code)

{'KOREA': 82, 'CHINA': 86, 'INDIA': 91, 'UNITED STATES': 1, 'INDONESIA': 62, 'BRAZIL': 55, 'PAKISTAN': 92, 'BANGLADESH': 880, 'NIGERIA': 234, 'RUSSIA': 7}


## 3.3 공통적인 매핑 메서드

다음 표는 `dict, collections.defaultdict, collections.OrderedDict`에서 제공하는 메서드를 비교해놓은 것이다.

| 메서드                       | dict | defaultdict | OrderedDict | 설명                                                         |
| ---------------------------- | ---- | ----------- | ----------- | ------------------------------------------------------------ |
| `d.clear()`                  | O    | O           | O           | 모든 항목 제거                                               |
| `d.__contains__(k)`          | O    | O           | O           | `k in d`                                                     |
| `d.copy()`                   | O    | O           | O           | 복사                                                         |
| `d.__copy__()`               |      | O           |             | `copy.copy()` 지원                                           |
| `d.default_factory`          |      | O           |             | 빠진값을 설정하기 위한 객체                                  |
| `d.__delitem__(k)`           | O    | O           | O           | `del d[k]`: 키가 `k` 인 항목 제거                            |
| `d.fromkeys(it, [initial])`  | O    | O           | O           | iterator에 초기값(val)설정                                   |
| `d.get(k, [default])`        | O    | O           | O           | 키가 `k` 인 값을 반환                                        |
| `d.__getitem__(k)`           | O    | O           | O           | 키가 `k` 인 값을 반환                                        |
| `d.items()`                  | O    | O           | O           | (key, val) 쌍으로 반환                                       |
| `d.__iter__()`               | O    | O           | O           | key에 대한 iterator 반환                                     |
| `d.keys()`                   | O    | O           | O           | key에 대한 view를 가져옴                                     |
| `d.__len__()`                | O    | O           | O           | item 개수 반환                                               |
| `d.__missing__(k)`           |      | O           |             | `__getitem__()` 이 `k`  키를 찾을 수 없을 때 호출            |
| `d.move_to_end(k, [last])`   |      |             | O           | 앞이나 뒤에서 k개의 item을 이동                              |
| `d.pop(k, [default])`        | O    | O           | O           | `k` 키 항목을 제거하고 반환                                  |
| `d.popitem()`                | O    | O           | O           | 처음이나 마지막 (key, val)을 제거하고 반환                   |
| `d.__reserved__()`           |      |             | O           | key에 대해 역순 iterator 가져옴                              |
| `d.setdefault(k, [default])` | O    | O           | O           | `k in d` 가 `True` 일 경우 `d[k]` 를 반환, `False`  일 경우 `d[k] = default` 로 설정 후 반환 |
| `d.__setitem__(k, v)`        | O    | O           | O           | `d[k] = v` 로 설정                                           |
| `d.update(m, [**kargs])`     | O    | O           | O           | (key, val) 형태의 `m` 으로 `d` 업데이트                      |
| `d.values()`                 | O    | O           | O           | value에 대한 view를 가져옴                                   |


### 3.3.1 존재하지 않는 키를 setdefault()로 처리하기

`dict`는 존재하지 않는 key로 접근하면 `KeyError`라는 에러가 나타난다. 이러한 에러를 방지하고자 `dict` 는 `.setdefault()`라는 메서드를 제공한다. `setdefault()`는 위의 표에서도 확인할 수 있듯이, key가 존재하지 않는 경우 default값으로 `d[k] = default`로 설정한 후 반환해준다.

아래의 예제는 존재하지 않는 key인 `d`에 `.setdefault()`메서드를 이용해 `0`으로 초기화하는 예제이다.

In [16]:
d = {'a': 1, 'b': 2, 'c': 3}
d.setdefault('d', 0)

print(d)

{'a': 1, 'b': 2, 'c': 3, 'd': 0}


## 3.4 Mappings with flexible key lookup

### 3.4.1 defaultdict: 존재하지 않는 키에 대한 또 다른 처리

3.3.1의 `setdefault()` 메서드를 사용하는 방법 말고, `collections`모듈의 `defaultdict`를 이용해 존재하지 않는 키(key)에 대해 처리해줄 수 있다. 

`collections.defaultdict(default_factory, key=value,...)` 에서 `default_factory` 인자에 초기값을 설정해주면 된다. `default_factory`인자는 메소드 형태의 값을 인자로 받는데, `list(), int(), set()...`나 사용자가 직접 메소드를 생성할 수 있다.

아래의 예제와 같이 `defaultdict`를 정의해줄 때 `int`로 지정해줬을 때 존재하지 않는 key인 `d`에 대한 초기값과 `default_factory` 메서드를 직접만들어 줬을 때 `d`에 대한 초기값을 보여준다.

In [75]:
from collections import defaultdict

# 1. int로 설정했을 때
d = defaultdict(int, a=1, b=2, c=3)
print("d['d'] =", d['d'])


# 2. default_factory를 직접 만들었을 때
def default_factory():
    return 'null'

d = defaultdict(default_factory, a=1, b=2, c=3)
print("d['d'] =", d['d'])

d['d'] = 0
d['d'] = null


### 3.4.2 __missing__() 메서드

또 다른 방법으로는 `__missing__()` 메서드를 이용해 직접 존재하지 않는 key에 대해 처리해 줄 수 있다. 아래의 예제는 `MyDict`라는 `dict`을 상속받은 클래스에서 `__missing__()` 메서드를 이용해 존재하지 않는 key를 처리해 주는 예제이다.

In [77]:
class MyDict(dict):
    
    def __missing__(self, key):
        return 'null'
    
    def get(self, key, default=None):
        try:
            return self[key]
        except KeyError:
            return default
        
    def __contains__(self, key):
        return key in self.keys() or str(key) in self.keys()

In [78]:
d = MyDict(a=1, b=2, c=3)

print("d['d'] =", d['d'])

d['d'] = null


## 3.5 UserDict 상속하기

`collections.UserDict`는 `dict`처럼 동작하는 매핑을 구현한 클래스이다. 위의 3.4.2 예제에서 메서드들의 오버라이드 문제 때문에 `dict`를 상속받는거 보다는 `UserDict`를 상속받는 것이 더 낫다고 한다. 또한, `UserDict`는 `data`라는 `dict` 객체를 가지고 있어 `__setitem__()` 과 `__contains__()` 메서드를 간단하게 구현할 수 있다. 

아래의 예제는 `collections.UserDict`를 이용해 3.4.2의 `MyDict`를 구현한 것이다. 

In [79]:
from collections import UserDict

class MyDict(UserDict):
    
    def __missing__(self, key):
        return 'null'
    
    def __contains__(self, key):
        return str(key) in self.data
    
    def __setitem__(self, key, item):
        self.data[str(key)] = item

In [81]:
d = MyDict(a=1, b=2, c=3)

print("d['d'] =", d['d'])
print('d.data :', d.data)

d['d'] = null
d.data : {'a': 1, 'b': 2, 'c': 3}


## 3.7 불변 매핑

`Mapping`의 타입은 전부 가변형이어서, 사용자가 데이터를 조작할 수 있다. 하지만, `types.MappingProxyType` 를 사용하면 읽기 전용(read-only)이기 때문에 불변형(immutable)이 된다.

`types.MappingProxyType`은 위의 설명대로 읽기 전용의 `mappingproxy` 객체를 반환한다. 아래의 예제는 `mappingproxy`를 사용하는 방법을 보여주는 예제이다.

In [84]:
from types import MappingProxyType

d = {'a': 1}

d_proxy = MappingProxyType(d)
print('d_proxy :', d_proxy)

# d_proxy에 (key, val)을 추가
d_proxy['b'] = 2

d_proxy : {'a': 1}


TypeError: 'mappingproxy' object does not support item assignment

위의 출력결과에서 확인할 수 있듯이, `MappingProxyType`인 `d_proxy`에 키 `b`를 추가할 경우 에러가 난다. 하지만, 아래의 코드처럼 직접 `d`에 키 `b`를 추가하면 `d_proxy`에 반영된 것을 확인할 수 있다.

In [85]:
# d에 (key, val) 추가
d['b'] = 2

print('d_proxy :', d_proxy)

d_proxy : {'a': 1, 'b': 2}
